In [ ]:
import os
import numpy as np
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

TRAINING_SET_DIR = os.getcwd() + '/drive/MyDrive/dog_pics/all_breeds_augmented/training_set_2/'
VALIDATION_SET_DIR = os.getcwd() + '/drive/MyDrive/dog_pics/all_breeds_augmented/validation_set/'

IMAGE_WIDTH = 500
IMAGE_HEIGHT = 375
dog_pics_training = 188
dog_pics_validation = 50

def run_model():
    # Use InceptionV3 model for feature extraction
    inception = InceptionV3(weights='imagenet', include_top=False)
    inception_output = inception.output
    inception_output = GlobalAveragePooling2D()(inception_output)

    # Add fully connected layer and output (prediction/classification) layer
    fc_layer = Dense(units=480, activation ='relu')(inception_output)
    predictions = Dense(120, activation ='softmax')(fc_layer)
    model = Model(inputs = inception.input, outputs = predictions)

    # Freeze InceptionV3 layers
    for layer in inception.layers:
        layer.trainable = False

    # Summarise and compile model
    model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    # Generate training and validation data with augmentations
    gen_training = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=360)
    training_data = gen_training.flow_from_directory(
        directory=TRAINING_SET_DIR,
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        class_mode='categorical',
    )

    gen_validation = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=360)
    validating_data = gen_validation.flow_from_directory(
        directory=VALIDATION_SET_DIR,
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        class_mode='categorical',
    )

    # Fit model
    model.fit_generator(
        training_data,
        steps_per_epoch=training_data.n // training_data.batch_size,
        epochs=5,
        validation_data=validating_data,
        validation_steps=validating_data.n // validating_data.batch_size
    )


    # Visualizing layers to see which to freeze. (Winging it with keras tutorial)
    for i, layer in enumerate(inception.layers):
      print(i, layer.name)

    # we chose to train the top 2 inception blocks, i.e. we will freeze
    # the first 249 layers and unfreeze the rest:
    for layer in model.layers[:249]:
      layer.trainable = False
    for layer in model.layers[249:]:
      layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        training_data,
        steps_per_epoch=training_data.n // training_data.batch_size,
        epochs=10,
        validation_data=validating_data,
        validation_steps=validating_data.n // validating_data.batch_size
    )
